In [36]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
from numba import jit
from scipy import interpolate
from scipy import integrate
from scipy import special

In [37]:
# common, fixed parameters:
#Morse parameters:
D=0.475217 # well depth in Hartree
alpha=1.17199002  # anharmonicity
mu=12178.1624678 # reduced mass
omega=math.sqrt(2.0*D*alpha**2.0/mu) # parameterized to follow CO stretch frequency

adiabatic_gap=3.0/27.2114 # energy difference between 2 surfaces
sigma_sq=(0.2*omega)*(0.1*omega) # gaussian broadening parameter, 2/10 of frequency
max_v=25 # maximum vibrational frequency considered

num_points=200  # number of points over which Morse Wfn is integrated

N_samples=500
x=np.linspace(adiabatic_gap-5.0*omega,adiabatic_gap+20.0*omega,N_samples)


In [38]:
# Define functions:
def zero_temp_FC_fac(Delta_sq,v):
    return 1.0/np.math.factorial(v)*np.power(Delta_sq/2.0,v)*np.exp(-Delta_sq/2.0)

def gaussian_func(sigma_sq,shift,x):
    return 1.0/np.sqrt(2.0*np.pi*sigma_sq)*np.exp(-np.power(x-shift,2)/(2.0*sigma_sq))

# full FC spectrum
def zero_temp_FC_sum(Delta_sq,adiabatic_gap,omega,sigma_sq,max_v,x):
    y_val=np.zeros(x.shape[0])
    for i in range(max_v):
        y_val=y_val+zero_temp_FC_fac(Delta_sq,i)*gaussian_func(sigma_sq,adiabatic_gap+omega*i*1.0,x)
    return y_val

# full Morse sepctrum:
def zero_temp_Morse_sum(Delta_sq,num_points,D,alpha,mu,omega,sigma_sq,max_v, adiabatic_gap,x):
    eff_displacement=Delta_sq
    y_val=np.zeros(x.shape[0])
    fc_facs,energies=compute_morse_fc_facs_energies(num_points,D,alpha,mu,omega,eff_displacement,max_v)
    for i in range(max_v):
        y_val=y_val+fc_facs[i]*fc_facs[i]*gaussian_func(sigma_sq,energies[i]+adiabatic_gap,x)
    return y_val

# functions for Morse osc
@jit(fastmath=True)
def compute_morse_eval_n(omega,D,n):
    return omega*(n+0.5)-(omega*(n+0.5))**2.0/(4.0*D)
  
@jit(fastmath=True)
def morse_eval_diff(omega,D,n):
    return compute_morse_eval_n(omega,D,n)-compute_morse_eval_n(omega,D,0)

# compute wavefunction of morse oscillator.
def compute_wavefunction_n(num_points, start_point,end_point,D,alpha,mu,n,shift):
    # first start by filling array with position points:
    wavefunc=np.zeros((num_points,2))
    lamda=math.sqrt(2.0*D*mu)/(alpha)
    step_x=(end_point-start_point)/num_points
    denom=special.gamma(2.0*lamda-n)
    if np.isinf(denom):
        denom=10e280
    num=(math.factorial(n)*(2.0*lamda-2.0*n-1.0))
    normalization=math.sqrt(num/denom)
    counter=0
    for x in wavefunc:
        x[0]=start_point+counter*step_x
        r_val=(start_point+counter*step_x)*alpha
        r_shift_val=(shift)*alpha
        z_val=2.0*lamda*math.exp(-(r_val-r_shift_val))
        func_val=normalization*z_val**(lamda-n-0.5)*math.exp(-0.5*z_val)*special.assoc_laguerre(z_val,n,2.0*lamda-2.0*n-1.0)
        x[1]=func_val
        counter=counter+1

    # fix normalization regardless of value of denominator to avoid rounding errors
    wavefunc_sq=np.zeros(wavefunc.shape[0])
    wavefunc_sq[:]=wavefunc[:,1]*wavefunc[:,1]

    normalization=integrate.simps(wavefunc_sq,dx=step_x)
    for counter in range(wavefunc.shape[0]):
        wavefunc[counter,1]=wavefunc[counter,1]/math.sqrt(normalization)

    return wavefunc

def find_classical_turning_points_morse(max_v,omega,alpha,D,Delta_sq):
        E_max_gs=compute_morse_eval_n(omega,D,max_v) # compute the energies for the highest energy morse 
        E_max_ex=compute_morse_eval_n(omega,D,max_v) # state considered

        # find the two classical turning points for the ground state PES
        point1_gs=math.log(math.sqrt(E_max_gs/D)+1.0)/(-alpha)
        point2_gs=math.log(-math.sqrt(E_max_gs/D)+1.0)/(-alpha)

        # same for excited state. Include shift vector
        point1_ex=math.log(math.sqrt(E_max_ex/D)+1.0)/(-alpha)+Delta_sq
        point2_ex=math.log(-math.sqrt(E_max_ex/D)+1.0)/(-alpha)+Delta_sq

        # now find the smallest value and the largest value
        start_point=min(point1_gs,point2_gs)
        end_point=max(point1_ex,point2_ex)

        return start_point,end_point

def compute_morse_fc_facs_energies(num_points,D,alpha,mu,omega,Delta_sq,max_v):
    fc_facs=np.zeros(max_v)
    energies=np.zeros(max_v)
    start_point,end_point=find_classical_turning_points_morse(max_v,omega,alpha,D,Delta_sq)
    gs_func=compute_wavefunction_n(num_points, start_point,end_point,D,alpha,mu,0,0.0)
    for i in range(max_v):
        ex_func=compute_wavefunction_n(num_points, start_point,end_point,D,alpha,mu,i,Delta_sq)
        tot_func=ex_func
        tot_func[:,1]=ex_func[:,1]*gs_func[:,1]
        fc_facs[i]=integrate.simps(tot_func[:,1],dx=tot_func[1,0]-tot_func[0,0])
        energies[i]=morse_eval_diff(omega,D,i)
    return fc_facs, energies



In [39]:
# define plotting function Franck-Condon harmonic:
def plt_zero_temp_FC(Displacement):
    # Delta_sq=mu*omega/(2pi*hbar)*displacement^2
    y=zero_temp_FC_sum(Displacement**2.0*omega*mu,adiabatic_gap,omega,sigma_sq,max_v,x)
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.set(xlabel='Energy (eV)', ylabel='Absorbance (arb. units)')
    fig.set_size_inches(14, 14)
    ax1.plot(x*27.2114,y,c='k',lw=3,label='Zero temperature spectrum')
    ax1.legend()
    q_vals=np.linspace(-1.0,1.0,N_samples)
    V_gs=1.0/2.0*mu*omega**2.0*(q_vals)**2.0
    V_ex=adiabatic_gap+1.0/2.0*mu*omega**2.0*(q_vals-Displacement)**2.0
    
    ax2.set(xlabel='Displacement (Ang)', ylabel='Energy (eV)')
    ax2.plot(q_vals*0.529177,V_gs*27.2114,lw=3,label='V_gs')
    ax2.plot(q_vals*0.529177,V_ex*27.2114,lw=3,label='V_ex')
    plt.legend()
    
    return y

# define plotting function Franck-Condon harmonic:
def plt_zero_temp_Morse(Displacement):
    y=zero_temp_Morse_sum(Displacement,num_points,D,alpha,mu,omega,sigma_sq,max_v, adiabatic_gap,x)
    y2=zero_temp_FC_sum(Displacement**2.0*omega*mu,adiabatic_gap,omega,sigma_sq,max_v,x)
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.set(xlabel='Energy (eV)', ylabel='Absorbance (arb. units)')
    fig.set_size_inches(14, 14)
    ax1.plot(x*27.2114,y,c='r',lw=3, label='Zero temperature Morse')
    ax1.plot(x*27.2114,y2,c='k',lw=3, label='Zero temperature Harmonic')
    ax1.legend()
    q_vals=np.linspace(-0.4,0.8,N_samples)
    V_gs=1.0/2.0*mu*omega**2.0*(q_vals)**2.0
    V_gs_morse=D*(1.0-np.exp(-alpha*q_vals))**2.0
    V_ex_morse=adiabatic_gap+D*(1.0-np.exp(-alpha*(q_vals-Displacement)))**2.0
    V_ex=adiabatic_gap+1.0/2.0*mu*omega**2.0*(q_vals-Displacement)**2.0
    
    ax2.set(xlabel='Displacement (Ang)', ylabel='Energy (eV)')
    ax2.plot(q_vals*0.529177,V_gs*27.2114,lw=3,label='V_gs_harmonic')
    ax2.plot(q_vals*0.529177,V_ex*27.2114,lw=3,label='V_ex_harmonic')
    ax2.plot(q_vals*0.529177,V_gs_morse*27.2114,'--',lw=3,label='V_gs_morse')
    ax2.plot(q_vals*0.529177,V_ex_morse*27.2114,'--',lw=3,label='V_ex_morse')
    ax2.legend()
    
    return y

# Electronic absorption spectrum for a simple displaced harmonic oscillator at T=0

At zero temperature, only the lowest vibrational level on the ground state potential energy surface is occupied and the electronic absorption spectrum reduces to a simple sum over final states. 

$I(\omega)\propto \omega|\mu_{01}|^2\sum_{v'}|\langle 0|v'\rangle|^2\delta(\omega-(E_{v'}-E_{v=0})/\hbar)$

The Franck-Condon integrals for the harmonic oscillator can be evaluated analytically, such that the spectrum can be written as:

$I(\omega)\propto \omega|\mu_{01}|^2\sum_{v'}\frac{1}{v'!}\left(\frac{\Delta^2}{2}\right)^{v'}e^{-\Delta^2/2}\delta(\omega-(E_{v'}-E_{v=0})/\hbar)$

where $\Delta\propto \sqrt{\omega_0}(R_e-R_e')$.

Below you find the absorption spectrum for a displaced harmonic oscillator modelled after the ground state PES of CO.
* How does the spectrum change with increasing displacement?
* What happens at very large displacement? Is this realistic? 



In [40]:
# now start interactive plotting
y=interactive(plt_zero_temp_FC,Displacement=(0.0,0.35,0.01))
display(y)

interactive(children=(FloatSlider(value=0.17, description='Displacement', max=0.35, step=0.01), Output()), _do…

# The role of anharmonicity: The displaced Morse oscillator

Below you find the absorption spectrum for a displaced Morse oscillator modelled after the ground state PES of CO, in comparison with the corresponding harmonic oscillator.

* What are the differences between the Morse and the harmonic spectrum? How are they related to the shape of the PES and the displacement?
* What happens to the Morse spectrum at large displacement?

In [41]:
y=interactive(plt_zero_temp_Morse,Displacement=(0.0,0.35,0.01))
display(y)

interactive(children=(FloatSlider(value=0.17, description='Displacement', max=0.35, step=0.01), Output()), _do…

In [42]:
# exact lineshape function in the 2nd order cumulant approximation for a single mode with equal ground and excited 
# state harmonic frequencies
def response_func(K,omega,kbT,adiabatic_gap,sigma_sq,t_vals): 
    n=1.0/(np.exp(omega/kbT)-1.0)
    m=n+1.0
    chi=np.zeros((t_vals.shape[0],2),dtype=np.complex_)
    g=np.zeros(t_vals.shape[0],dtype=np.complex_)
    av_energy_gap=adiabatic_gap+1.0/2.0*(K*omega)**2.0
    for i in range(t_vals.shape[0]):
        g=K**2.0*omega/(2.0)*(2.0*n+1.0-1j*omega*t_vals[i]-m*cmath.exp(-1j*omega*t_vals[i])-n*cmath.exp(1j*omega*t_vals[i]))
        chi[i,1]=cmath.exp(-g)*cmath.exp(-sigma_sq*t_vals[i]**2.0/2.0)
        chi[i,0]=t_vals[i]
    return chi,av_energy_gap

@jit
def full_spectrum_integrant(chi,E_val):
    integrant=np.zeros(chi.shape[0])
    counter=0
    while counter<integrant.shape[0]:
        integrant[counter]=(chi[counter,1]*cmath.exp(1j*chi[counter,0]*E_val)).real
        counter=counter+1
    return integrant

def finite_temp_fc_spec(T,K,omega,adiabatic_gap,sigma_sq,x):
    kbT=T*8.6173303*10.0**(-5.0)/27.211396132 # kbT in hartree
    max_t=300.0/(2.418884326505*10.0**(-2.0)) # 500 fs in hartree
    t_vals=np.linspace(0.0,max_t,num_points*10)
    chi,av_energy_gap=response_func(K,omega,kbT,adiabatic_gap,sigma_sq,t_vals)
    spectrum=np.zeros(x.shape[0])
    i=0
    for energy in x:
        integrant=full_spectrum_integrant(chi,energy-av_energy_gap)
        spectrum[i]=integrate.simps(integrant,dx=chi[1,0].real-chi[0,0].real)
        i=i+1
    return spectrum



In [43]:
def plt_finite_temp_FC(Displacement,Temperature):
    # Delta_sq=mu*omega/(2pi*hbar)*displacement^2
    y=finite_temp_fc_spec(Temperature,Displacement*110.0,omega,adiabatic_gap,sigma_sq,x)
    y2=zero_temp_FC_sum(Displacement**2.0*omega*mu,adiabatic_gap,omega,sigma_sq,max_v,x)
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.set(xlabel='Energy (eV)', ylabel='Absorbance (arb. units)')
    fig.set_size_inches(14, 14)
    l1=ax1.plot(x*27.2114,y/math.sqrt(2.0*math.pi)*0.8,c='r',lw=3,label='Finite temperature')
    l2=ax1.plot(x*27.2114,y2,lw=3,c='k', label='Zero temperature')
    ax1.legend()
    q_vals=np.linspace(-1.0,1.0,N_samples)
    V_gs=1.0/2.0*mu*omega**2.0*(q_vals)**2.0
    V_ex=adiabatic_gap+1.0/2.0*mu*omega**2.0*(q_vals-Displacement)**2.0
    
    ax2.set(xlabel='Displacement (Ang)', ylabel='Energy (eV)')
    ax2.plot(q_vals*0.529177,V_gs*27.2114,lw=3,label='V_gs')
    ax2.plot(q_vals*0.529177,V_ex*27.2114,lw=3,label='V_ex')
    ax2.legend()
    
    return y

# Temperature effects in electronic absorption spectra

To include temperature effects, we have to expand our sum over final states to a sum over both intital and final states, with a Boltzmann weighting for the initial states. 

$I(\omega)\propto \omega|\mu_{01}|^2\sum_ve^{-\frac{E_v}{k_\textrm{B}T}}\sum_{v'}|\langle v|v'\rangle|^2\delta(\omega-(E_{v'}-E_v)/\hbar)$

* Is the zero temperature approximation a good approximation for CO?
* What is the effect of temperature at low values of T?
* What happens to the spectrum at very high values of T? 

In [44]:
y=interactive(plt_finite_temp_FC,Displacement=(0.0,0.3,0.01),Temperature=(10,5000,50))
display(y)

interactive(children=(FloatSlider(value=0.15, description='Displacement', max=0.3, step=0.01), IntSlider(value…